In [15]:
import requests
import pandas as pd
import time
import urllib.parse
from bs4 import BeautifulSoup

In [16]:
# Search settings (edit here)
num_google_search = 30 # Number of queries for each search term/context pair (min: 1, max: 99)
contexts = ['Permian', 'Midland'] # context to be paired with search terms
search_terms = ['horizontal', 'oil', 'Chevron'] # search terms

In [45]:
req = requests.get('https://na.op.gg/summoner/userName=gffgfgfg')
soup = BeautifulSoup(req.text, "html.parser")
#print(soup.prettify())

for link in soup.find_all("div", {"class":"GameItem Win "}):
    print(link.attrs['data-game-id'])
    
for link in soup.find_all("div", {"class":"GameItem Lose "}):
    print(link.attrs['data-game-id'])




2589045438
2588996588
2588984682
2588725744
2588678106
2588528995
2588504870
2588369073
2588364186
2588351262
2587770963
2587602981
2587529020
2589031365
2588583470
2588218325
2587568600
2587553212
2587515562


In [3]:
# get content from a seeking alpha article
def mine_seeking_alpha(url):
    try:      
        req = requests.get(url.strip(), timeout=5)
        soup = BeautifulSoup(req.text, "html.parser")
    except:
        return None
    
    title = ""
    date = ""
    author = ""
    content = ""
    
    # Get article title
    try:
        title = soup.find("h1", {"itemprop":"headline"}).text.strip()
    except:
        title = ""
    # Get article date
    try:
        date = soup.find("time", {"itemprop":"datePublished"})['content'].strip()
    except:
        date = ""
    # Get author name
    try:
        author = soup.find("a", {"rel":"author"}).find("span").text.strip()
    except:
        author = ""
    # Get content
    try:
        content = soup.find("div", {"id":"a-cont"}).get_text(' ').strip()
    except:
        content = ""
    # Get summary
    try:
        summary = soup.find("div", {"class":"article-summary"}).get_text(' ').strip()
    except:
        summary = ""
    
    # Get tickers
    try:
        tickers = []
        links = soup.find_all("a")
        for link in links:
            if '/symbol/' in link['href']: # looking for symbol link, as those are what we want
                tickers.append(link['href'].split("/symbol/",1)[1] ) # getting symbol name from the link
        tickers = list(set(tickers)) # remove any duplicate tickers for a single article
    except:
        tickers = ""
    
    if not content:
        return
    
    return pd.Series({
        "url": url,
        "date": date[:-10],
        "title": title,
        "author": author,
        "content": content,
        "tickers": tickers,
        "summary": summary
    })

In [4]:
data = pd.DataFrame()

In [5]:
queries = len(contexts) * len(search_terms) * num_google_search
current_query = 0

print("Progress (%):")

for context in contexts:
    for search_term in search_terms:
        
        # Get search results and article content from each combination of search term & context.
        
        # url safe encode the context and search term
        req = requests.get('https://www.google.com/search?q=' + urllib.parse.quote_plus(context) + '+%20+' + urllib.parse.quote_plus(search_term) + 'site%3Aseekingalpha.com&start=0&num=' + str(num_google_search), timeout=15)
        soup = BeautifulSoup(req.text, "html.parser")
        
        # Get Search Results fom Google
        search_results = []
        for h3 in soup.find_all("h3", {"class":"r"}): # All search links are in h3 tags
            url = h3.find("a")['href'][7:] # Get link & remove google query at the beginning of the link
            url = url.split('&', 1)[0] # Get rid of ending GET variables from the article link... this causes a 403 Error (Forbidden) to occur
            if url[:4] == 'http': # checking for valid links (not more google searches or other google queries)
                search_results.append(url)
        
        queries = queries - (num_google_search - len(search_results))
        # Go through each seeking alpha article. Look for content, title, date, etc.
        for search_result in search_results:
            current_query += 1
            print(current_query/queries*100)
            article = mine_seeking_alpha(search_result)
            if article is not None:
                article = article.append(
                pd.Series({
                        "context": str(context),
                        "search term": str(search_term)
                    })
                )
            data = data.append(article, ignore_index=True)
print(current_query)            

Progress (%):
0.5555555555555556
1.1111111111111112
1.6666666666666667
2.2222222222222223
2.7777777777777777
3.3333333333333335
3.888888888888889
4.444444444444445
5.0
5.555555555555555
6.111111111111111
6.666666666666667
7.222222222222221
7.777777777777778
8.333333333333332
8.88888888888889
9.444444444444445
10.0
10.555555555555555
11.11111111111111
11.666666666666666
12.222222222222221
12.777777777777777
13.333333333333334
13.88888888888889
14.444444444444443
15.0
15.555555555555555
16.11111111111111
16.666666666666664
17.71428571428571
18.285714285714285
18.857142857142858
19.428571428571427
20.0
20.57142857142857
21.142857142857142
21.714285714285715
22.285714285714285
22.857142857142858
23.42857142857143
24.0
24.571428571428573
25.142857142857146
25.71428571428571
26.285714285714285
26.857142857142858
27.42857142857143
28.000000000000004
28.57142857142857
29.142857142857142
29.714285714285715
30.28571428571429
30.857142857142854
31.428571428571427
36.36363636363637
37.012987012987

In [6]:
# remove duplicate articles based on having the same url
data = data.drop_duplicates(subset=['url'])
data = data.sort_values('date', ascending=False).reset_index(drop=True)

In [7]:
data.sort_values('date', ascending=False)

,author,content,context,date,search term,summary,tickers,title,url
0,Laurentian Research,"Summary During the Q2 2017 reporting season, G...",Permian,2017-08-09,horizontal,"Summary During the Q2 2017 reporting season, G...","[PXD, REN]",Understanding GOR In Unconventional Play: Perm...,https://seekingalpha.com/article/4096835-under...
1,Elliott Gue,Summary US short-cycle plays appear poised to ...,Permian,2017-07-06,horizontal,Summary US short-cycle plays appear poised to ...,"[EOG, ETP, PXD, AMLP, XOM, WPZ, APA, SLB, BHGE...",Permian Basin To Take Market Share In Short-Cy...,https://seekingalpha.com/article/4086117-permi...
2,Laurentian Research,Summary A list of 78 transactions on Permian B...,Permian,2017-06-28,horizontal,Summary A list of 78 transactions on Permian B...,"[EOG, PXD, XOM, AXAS, REN, APA, BP, CEO, JAG, ...",Permian Basin: Is The Land Grab Turning Into P...,https://seekingalpha.com/article/4084552-permi...
3,Laurentian Research,Summary We've compiled a list of mostly public...,Permian,2017-06-27,horizontal,Summary We've compiled a list of mostly public...,"[EOG, PXD, XOM, HES, AREX, REN, XEC, COP, APA,...",Competitive Landscape In The Permian Basin,https://seekingalpha.com/article/4084347-compe...
4,Jennifer Warren,Summary The Permian Basin will keep growing pr...,Permian,2017-06-22,horizontal,Summary The Permian Basin will keep growing pr...,"[CVS, ECA, EOG, APC, PXD, XOM, RSPP, F, USO, X...","U.S. Oil, The Permian, And Global Outlook",https://seekingalpha.com/article/4083250-u-s-o...
5,Randy Durig,Summary Increased oil production quarter over ...,Permian,2017-06-22,horizontal,Summary Increased oil production quarter over ...,[LGCY],"Legacy Reserves: Over 20% Yield, Maturing Dece...",https://seekingalpha.com/article/4083098-legac...
6,Daniel Sermersheim,"Summary ROE, ROA, margins, sales, and net inco...",Midland,2017-04-11,horizontal,"Summary ROE, ROA, margins, sales, and net inco...",[SMID],Smith Midland Is Fundamentally Positioned For ...,https://seekingalpha.com/article/4061726-smith...
7,GrowthHunter,"Summary 1,572 drilling permits approved during...",Permian,2017-04-06,horizontal,"Summary 1,572 drilling permits approved during...","[ECA, EOG, APC, PXD, WPX, EPE, EGN, PE, XOM, H...",Permian Drilling Permits: Q1 2017 Review,https://seekingalpha.com/article/4060747-permi...
8,Laura Starks,Summary Diamondback Energy has differentiated ...,Permian,2017-04-04,horizontal,Summary Diamondback Energy has differentiated ...,"[FANG, AREX]",Diamondback Energy Strikes Oil Again And Again,https://seekingalpha.com/article/4060109-diamo...
9,BTU Analytics,Summary Ever since hitting the bottom in July ...,Permian,2017-03-15,horizontal,Summary Ever since hitting the bottom in July ...,"[UCO, BNO, USL, SCO, OILK, OIL, OILX, DTO, DBO...",Drilling Activity: How Much Does The Market Need?,https://seekingalpha.com/article/4055446-drill...


In [9]:
# Export article extracts to csv
data.to_csv('articles.csv')
# *** UNCOMMENT ABOVE LINE ***